NEW Code
1. Code to convert pdf to excel sheet with hyperlinks


In [80]:
# import pandas as pd
# import pdfplumber
# import fitz  # PyMuPDF
# import re
# 
# def extract_links_from_pdf(pdf_path):
#     """
#     Extract all hyperlinks from PDF with their positions
#     """
#     doc = fitz.open(pdf_path)
#     all_links = []
#     
#     for page in doc:
#         links = page.get_links()
#         for link in links:
#             if 'uri' in link:
#                 rect = link['from']
#                 uri = link['uri']
#                 y_pos = rect[1]
#                 all_links.append({
#                     'uri': uri,
#                     'y_pos': y_pos,
#                     'page': page.number
#                 })
#     
#     doc.close()
#     return all_links
# 
# def extract_table_with_positions(pdf_path):
#     """
#     Extract table content with text positions
#     """
#     table_data = []
#     
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             words = page.extract_words(keep_blank_chars=True)
#             
#             row_data = {
#                 'SL.NO': '',
#                 'NAME': '',
#                 'DESIGNATION': '',
#                 'Scopus_Link': None,
#                 'WOS_Link': None,
#                 'position': None
#             }
#             
#             for word in words:
#                 current_text = word['text'].strip()
#                 current_y = word['top']
#                 
#                 if current_text.replace('.', '').isdigit() and not row_data['SL.NO']:
#                     if len(row_data['SL.NO']) == 0:  # New row
#                         row_data['SL.NO'] = current_text
#                         row_data['position'] = current_y
#                 
#                 elif 'Profile Link' in current_text or current_text == 'NA':
#                     if row_data['Scopus_Link'] is None:
#                         row_data['Scopus_Link'] = current_text
#                     else:
#                         row_data['WOS_Link'] = current_text
#                         if row_data['SL.NO']:  # Only append if we have a valid row
#                             table_data.append(row_data.copy())
#                             row_data = {
#                                 'SL.NO': '',
#                                 'NAME': '',
#                                 'DESIGNATION': '',
#                                 'Scopus_Link': None,
#                                 'WOS_Link': None,
#                                 'position': None
#                             }
#                 
#                 elif row_data['SL.NO'] and not row_data['NAME']:
#                     row_data['NAME'] = current_text
#                 
#                 elif row_data['NAME'] and not row_data['DESIGNATION']:
#                     if 'Prof' in current_text or 'Head' in current_text:
#                         row_data['DESIGNATION'] = current_text
#                     else:
#                         row_data['NAME'] += ' ' + current_text
# 
#     return table_data
# 
# def map_links_to_rows(table_data, all_links):
#     """
#     Map extracted links to table rows based on position
#     """
#     scopus_links = [link for link in all_links if 'scopus' in link['uri'].lower()]
#     wos_links = [link for link in all_links if 'webofscience' in link['uri'].lower() or 'wos' in link['uri'].lower()]
#     
#     for row in table_data:
#         y_pos = row['position']
#         
#         # Find closest Scopus link
#         if row['Scopus_Link'] == 'Profile Link':
#             closest_scopus = min(scopus_links, key=lambda x: abs(x['y_pos'] - y_pos), default=None)
#             if closest_scopus:
#                 row['Scopus_Link'] = closest_scopus['uri']
#                 scopus_links.remove(closest_scopus)
#         
#         # Find closest WOS link
#         if row['WOS_Link'] == 'Profile Link':
#             closest_wos = min(wos_links, key=lambda x: abs(x['y_pos'] - y_pos), default=None)
#             if closest_wos:
#                 row['WOS_Link'] = closest_wos['uri']
#                 wos_links.remove(closest_wos)
# 
# def create_final_dataframe(table_data):
#     """
#     Create final DataFrame with mapped links
#     """
#     df = pd.DataFrame(table_data)
#     
#     # Remove the position column as it's no longer needed
#     df = df.drop('position', axis=1, errors='ignore')
#     
#     # Clean up the data
#     df['SL.NO'] = df['SL.NO'].str.replace('.', '').astype(int)
#     df = df.sort_values('SL.NO').reset_index(drop=True)
#     
#     # Clean text fields
#     df['NAME'] = df['NAME'].str.strip()
#     df['DESIGNATION'] = df['DESIGNATION'].str.strip()
#     
#     # Replace 'NA' text with None
#     df = df.replace('NA', None)
#     
#     return df
# 
# def process_pdf_with_links(pdf_path):
#     """
#     Main function to process PDF and create DataFrame with mapped links
#     """
#     # Extract links and their positions
#     all_links = extract_links_from_pdf(pdf_path)
#     
#     # Extract table data with positions
#     table_data = extract_table_with_positions(pdf_path)
#     
#     # Map links to table rows
#     map_links_to_rows(table_data, all_links)
#     
#     # Create final DataFrame
#     df = create_final_dataframe(table_data)
#     
#     return df
# 
# def export_to_excel(df, output_path):
#     """
#     Export DataFrame to Excel with clickable hyperlinks
#     """
#     writer = pd.ExcelWriter(output_path, engine='xlsxwriter')
#     df.to_excel(writer, index=False, sheet_name='Faculty List')
#     
#     workbook = writer.book
#     worksheet = writer.sheets['Faculty List']
#     
#     # Add hyperlink format
#     link_format = workbook.add_format({
#         'color': 'blue',
#         'underline': True,
#         'font_size': 11
#     })
#     
#     # Apply hyperlinks
#     for idx, row in df.iterrows():
#         if pd.notna(row['Scopus_Link']) and row['Scopus_Link'] != 'NA':
#             worksheet.write_url(idx + 1, 3, row['Scopus_Link'], link_format, 'Profile Link')
#         if pd.notna(row['WOS_Link']) and row['WOS_Link'] != 'NA':
#             worksheet.write_url(idx + 1, 4, row['WOS_Link'], link_format, 'Profile Link')
#     
#     # Adjust column widths
#     worksheet.set_column('A:A', 8)   # SL.NO
#     worksheet.set_column('B:B', 25)  # NAME
#     worksheet.set_column('C:C', 20)  # DESIGNATION
#     worksheet.set_column('D:E', 15)  # Links
#     
#     writer.close()
# 
# # Example usage
# 
# pdf_path = 'Resources/FACULTY-LIST-WITH-SCOPUS-WOS-PROFILE-LINK.pdf'
# 
# # Process PDF and create DataFrame
# df = process_pdf_with_links(pdf_path)
# 
# # Export to Excel with clickable links
# export_to_excel(df, 'Resources/faculty_list_with_actual_links.xlsx')
# 
# # Display first few rows
# print(df.head())


In [77]:
df

,SL.NO,NAME,DESIGNATION,Scopus_Link,WOS_Link
0,1,,,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
1,2,Dr. K. Paramesha,Professor,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
2,3,Dr. C Balarengadurai,Professor,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
3,4,Dr. Aditya C R,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
4,5,Dr. Natesh. M,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
5,6,Dr. Ramakrishna Hegde,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
6,7,Dr. Janhavi.V,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,https://www.webofscience.com/wos/author/record...
7,8,Dr. Ayesha Taranum,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,None
8,9,Dr. Prasad M R,Assoc. Prof.,https://www.scopus.com/authid/detail.uri?autho...,None
9,10,Dr. Hamsaveni M,Asst. Prof.,https://www.scopus.com/results/authorNamesList...,None


2. Collecting all the Scopus URL

In [78]:

# Initialize an empty list to store the URLs
URL = []

# Loop through the "Scopus Link" column and add each link to the URL list
for link in df['Scopus_Link']:
    URL.append(link)




['https://www.scopus.com/authid/detail.uri?authorId=57190388894', 'https://www.scopus.com/authid/detail.uri?authorId=57053407100', 'https://www.scopus.com/authid/detail.uri?authorId=57204777085', 'https://www.scopus.com/authid/detail.uri?authorId=57216440090', 'https://www.scopus.com/authid/detail.uri?authorId=57216492647', 'https://www.scopus.com/authid/detail.uri?authorId=56242461400', 'https://www.scopus.com/authid/detail.uri?authorId=57190803068', 'https://www.scopus.com/authid/detail.uri?authorId=57216610861', 'https://www.scopus.com/authid/detail.uri?authorId=56492215200', 'https://www.scopus.com/results/authorNamesList.uri?sort=count-f&src=al&affilName=Vidyavardhaka+College+of+Engineering&sid=ff00aae3a0aef9cbdce26caa697592a9&sot=al&sdt=al&sl=88&s=AUTHLASTNAME%28Hamsaveni%29+AND+AUTHFIRST%28M%29+AND+AFFIL%28Vidyavardhaka+College+of+Engineering%29&st1=Hamsaveni&st2=M&orcidId=&selectionPageSearch=anl&reselectAuthor=false&activeFlag=true&showDocument=false&resultsPerPage=20&offset=1

3. Sending Requests 

In [97]:

import requests

# Example user-agent (replace with your actual user-agent string)
user_agent = 'Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0'
link = "https://www.scopus.com/authid/detail.uri?authorId=57190388894"
    # Skip the link if it's 'None' or empty
# if link is None or link == "None":
#     continue  # Skip to the next iteration if the link is invalid

    # Make the request if the link is valid
response = requests.get(link, headers={'user-agent': user_agent})

    # Check if the request was successful
if response.ok:
    message = 'The request was successfully received, understood, and accepted.'
else:
    message = 'The request was not successful.'

    # Print the status code and message
print('Status Code:', response.status_code)
print(message)


Status Code: 200
The request was successfully received, understood, and accepted.


In [82]:
# import requests
# 
# # Example user-agent (replace with your actual user-agent string)
# #user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
# 
# for link in URL:
#     # Skip the link if it's 'None' or empty
#     if link is None or link == "None":
#         continue  # Skip to the next iteration if the link is invalid
#     
#     # Make the request if the link is valid
#     response = requests.get(link)
#     
#     # Check if the request was successful
#     if response.ok:
#         message = 'The request was successfully received, understood, and accepted.'
#     else:
#         message = 'The request was not successful.'
#     
#     # Print the status code and message
#     print('Status Code:', response.status_code)
#     print(message)


Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200
The request was successfully received, understood, and accepted.
Status Code: 200

In [117]:
from bs4 import BeautifulSoup

html = response.content
parser = 'html.parser'

soup = BeautifulSoup(html, parser)

In [118]:
print(soup)


<!DOCTYPE html>

<script>
isAuthorFeedbackMVP = true
</script>
<html lang="en_US">
<head>
<meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/>
<meta content="NO-CACHE" http-equiv="PRAGMA"/>
<meta content="M5q4CxMVJyf1WE5UwVmHfV8q3LK1vh/3qmo894V0Pqg=" name="verify-v1"/>
<meta content="1" name="tdm-reservation">
<meta content="https://www.elsevier.com/tdm/tdmrep-policy.json" name="tdm-policy">
<script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam-cell.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"31455944",accountID:"1281161",trustKey:"2038175",xpid:"VQQPUFdVCRADVVVXAwABVA==",licenseKey:"0268925da8",applicationID:"31454162"};;/*! For license information please see nr-loader-spa-1.273.0.min.js.LICENSE.txt */
(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)

In [119]:
table = soup.find('div')
print(table)
# <div style="--stack-vertical-spacing: var(--spacer-m); --stack-horizontal-spacing: var(--spacer-m); --stack-justify-content: flex-start; --stack-align-items: flex-start;" class="Stack-module__tT3r4 Stack-module___CTfk DocumentsHeader-module__k6SlX"><span class="Typography-module__lVnit Typography-module__ix7bs Typography-module__Nfgvc">31 documents</span></div>

<div class="container-fluid">
<div class="row-fluid mainRow" id="rowMain">
<div class="col-md-12 fullHeightHack content" id="contentContainer">
<header class="highlight-none" id="gh-cnt">
<a class="anchor gh-skip-navigation" href="#skip1"><span class="anchor-text">Skip to main content</span></a>
<a id="top" name="top"></a>
<input id="txGid" type="hidden" value="5F77FDAF79394FE95337CFD5CFA44432.i-05e916d0f887fe11a:4"/>
<script>
var prs = {rt: function(label,ts) { this[label]=(ts||ts==0?ts:new Date().getTime());}};
var continuePcrRecording = true;
var PCR_SCR_ATTRIBUTE_NAME = 'data-reportRenderTime';
function recordRendering (primaryContentElements, mutations) {
if (continuePcrRecording) {
primaryContentElements.forEach(function (element) {
for (var i = 0; i < mutations.length; i++) {
var changedNode = mutations[i].target;
if (element === changedNode || element.contains(changedNode)) {
var contentSignificance = element.getAttribute(PCR_SCR_ATTRIBUTE_NAME);
prs.rt(contentSignificance);
}


In [122]:

table2= table.find('div')
print(table2)


<div class="row-fluid mainRow" id="rowMain">
<div class="col-md-12 fullHeightHack content" id="contentContainer">
<header class="highlight-none" id="gh-cnt">
<a class="anchor gh-skip-navigation" href="#skip1"><span class="anchor-text">Skip to main content</span></a>
<a id="top" name="top"></a>
<input id="txGid" type="hidden" value="5F77FDAF79394FE95337CFD5CFA44432.i-05e916d0f887fe11a:4"/>
<script>
var prs = {rt: function(label,ts) { this[label]=(ts||ts==0?ts:new Date().getTime());}};
var continuePcrRecording = true;
var PCR_SCR_ATTRIBUTE_NAME = 'data-reportRenderTime';
function recordRendering (primaryContentElements, mutations) {
if (continuePcrRecording) {
primaryContentElements.forEach(function (element) {
for (var i = 0; i < mutations.length; i++) {
var changedNode = mutations[i].target;
if (element === changedNode || element.contains(changedNode)) {
var contentSignificance = element.getAttribute(PCR_SCR_ATTRIBUTE_NAME);
prs.rt(contentSignificance);
}
}
});
}
};
MutationObserver = 

In [124]:
table3 =table2.find('div')
print(table3)

<div class="col-md-12 fullHeightHack content" id="contentContainer">
<header class="highlight-none" id="gh-cnt">
<a class="anchor gh-skip-navigation" href="#skip1"><span class="anchor-text">Skip to main content</span></a>
<a id="top" name="top"></a>
<input id="txGid" type="hidden" value="5F77FDAF79394FE95337CFD5CFA44432.i-05e916d0f887fe11a:4"/>
<script>
var prs = {rt: function(label,ts) { this[label]=(ts||ts==0?ts:new Date().getTime());}};
var continuePcrRecording = true;
var PCR_SCR_ATTRIBUTE_NAME = 'data-reportRenderTime';
function recordRendering (primaryContentElements, mutations) {
if (continuePcrRecording) {
primaryContentElements.forEach(function (element) {
for (var i = 0; i < mutations.length; i++) {
var changedNode = mutations[i].target;
if (element === changedNode || element.contains(changedNode)) {
var contentSignificance = element.getAttribute(PCR_SCR_ATTRIBUTE_NAME);
prs.rt(contentSignificance);
}
}
});
}
};
MutationObserver = window.MutationObserver || window.WebKitMutat

In [103]:
company = first_row.find('div', {'class': 'Col-module_hwM1N'}).text
company

AttributeError: 'NoneType' object has no attribute 'text'

In [129]:
span = soup.find('span', class_=['Typography-module--h200--lVnit',
                                       'Typography-module--neutrals80--ix7bs',
                                       'Typography-module--weight-normal--Nfgvc'])
print(span)

None


In [131]:
t = soup.find('div', style="--stack-vertical-spacing: var(--spacer-m); --stack-horizontal-spacing: var(--spacer-m); --stack-justify-content: flex-start; --stack-align-items: flex-start;")

In [132]:
print(t)

None
